## Usar versão de python 3.10 ou maior

In [11]:
# Equipe:
# * Filipe Abner Soares Melo
# * Lucas Pereira Freitas
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET
import math
from itertools import cycle
from enum import Enum
from math import atan2
from copy import deepcopy
import numpy as np
import poligonos
from poligonos import Poligono
from poligonos import Reta
from poligonos import Ponto
from poligonos import Orientacao
from poligonos import Window
import reta_cohen
import reta_liang
from tkinter import simpledialog, messagebox

In [12]:
# Classe Visualizador -> Realiza todos os processos de visualização
class Visualizador:

    # Inicializa a window com suas coordenadas minimas e maximas (Será Lido do arquivo a posteriori)
    window = {
        "xmin" : 0,
        "ymin" : 0,

        "xmax" : 0,
        "ymax" : 0,
    }

    window_normalizada = {
        "xmin": 0,
        "ymin": 0,

        "xmax" : 0,
        "ymax" : 0,
    }

    # Inicializa a viewport com suas coordenadas minimas e maximas (Será Lido do arquivo a posteriori)
    viewport = {
        "xmin" : 0,
        "ymin" : 0,

        "xmax" : 0,
        "ymax" : 0,
    }

    # Inicializa a viewport do minimapa com suas coordenadas minimas e maximas
    minimap_viewport = {
        "xmin" : 0,
        "ymin" : 0,

        "xmax" : 150,
        "ymax" : 120,
    }

    # Inicializa o mundo com suas coordenadas minimas e maximas -> Necessário para representar o "Zoom Out" do minimapa
    world_size = {
        "xmin" : 0,
        "ymin" : 0,

        "xmax" : 50,
        "ymax" : 37.5,
    }

    # Inicializa a lista de formas e define se o arquivo foi aberto
    arquivo = False
    formas = list()
    calculou_window = False
    angulo = 0.0
    angulo_rotacao = 10.0
    escala = 1.0
    algClippingReta = -1

    # Inicializa a janela principal
    def __init__(self, root):
        self.root = root
        # self.root.bind("<KeyPress>", self.movimentar)
        self.root.title("Visualizador de Objetos 2D")

        # Configurar o menu
        menu = tk.Menu(root)
        root.config(menu=menu)
        file_menu = tk.Menu(menu)
        menu.add_cascade(label="Arquivo", menu=file_menu)
        file_menu.add_command(label="Abrir", command=self.abrir_arquivo)

        # Frame principal para conter canvas e minimapa
        frame_principal = tk.Frame(root)
        frame_principal.pack(fill="both", expand=True)

        # CARREGA BOTÕES
        frame_botoes = tk.Frame(root)
        frame_botoes.pack(pady=10)

        botao_cima = tk.Button(frame_botoes, text='⬆', command= lambda: self.movimentar("Up"))
        botao_cima.grid(row=0,column=1)

        botao_esquerda = tk.Button(frame_botoes, text='⬅', command= lambda: self.movimentar("Left"))
        botao_esquerda.grid(row=1,column=0)

        botao_baixo = tk.Button(frame_botoes, text='⬇', command= lambda: self.movimentar("Down"))
        botao_baixo.grid(row=1,column=1)

        botao_direita = tk.Button(frame_botoes, text='➡', command= lambda: self.movimentar("Right"))
        botao_direita.grid(row=1,column=2)

        botao_rotacionar_direita = tk.Button(frame_botoes, text='↩', command=lambda: self.rotacionar(angulo=self.angulo_rotacao))
        botao_rotacionar_direita.grid(row=1,column=10,padx=5)

        botao_rotacionar_esquerda = tk.Button(frame_botoes, text='↪', command=lambda: self.rotacionar(angulo=-self.angulo_rotacao))
        botao_rotacionar_esquerda.grid(row=1,column=11,padx=5)

        botao_aumentar_zoom = tk.Button(frame_botoes, text='➕', command=lambda: self.aplicar_zoom(0.10))
        botao_aumentar_zoom.grid(row=1,column=12,padx=5)

        botao_reduzir_zoom = tk.Button(frame_botoes, text='➖',command=lambda: self.aplicar_zoom(-0.10))
        botao_reduzir_zoom.grid(row=1,column=13,padx=5)

        # Define o tamanho da Viewport inicial
        largura = 100
        altura = 100

        # canvas da Viewport inicial
        self.canvas = tk.Canvas(frame_principal, width=largura, height=altura, bg="white")
        self.canvas.pack(side="left", fill="both", expand=True)

        # Canvas do minimapa na lateral direita
        # usar um tamanho fixo para o minimapa
        self.minimap = tk.Canvas(frame_principal, width=self.minimap_viewport['xmax'], height=self.minimap_viewport['ymax'], bg="lightgrey") # razão de aspecto 4:3
        self.minimap.pack(side="right", padx=10, pady=10)

    """
    Caixa de escolha do algoritmo de clipping para retas
    """
    def selecionar_algoritmo_clipping_retas(self):
        opcao = simpledialog.askstring(
            "...",
            "Algoritmo de clipping de reta:\n1. Cohen-Sutherland\n2. Liang-Barsky"
        )
        if opcao == "1":
            messagebox.showinfo("Algoritmo Selecionado", "Cohen-Sutherland")
            self.algClippingReta = 0
        elif opcao == "2":
            messagebox.showinfo("Algoritmo Selecionado", "Liang-Barsky")
            self.algClippingReta = 1
        else:
            messagebox.showwarning("Aviso","Nenhuma opção válida selecionada.\nPor padrão o algoritmo utilizado sera o de Liang-Barsky")
            self.algClippingReta = 1

    # Função para abrir o arquivo e Carregar as formas geometricas
    def abrir_arquivo(self):
        self.arquivo = True
        self.tree = ET.parse('entrada.xml')
        root = self.tree.getroot()
        vp = root.find('viewport')
        self.viewport['xmin'] = int(vp[0].attrib['x'])
        self.viewport['ymin'] = int(vp[0].attrib['y'])

        self.viewport['xmax'] = int(vp[1].attrib['x'])
        self.viewport['ymax'] = int(vp[1].attrib['y'])

        self.canvas.configure(height=int(self.viewport['ymax'] - self.viewport['ymin']),width=int(self.viewport['xmax'] - self.viewport['xmin']))

        window = root.find('window')

        self.window['xmin'] = float(window[0].attrib['x'])
        self.window['ymin'] = float(window[0].attrib['y'])

        self.window['xmax'] = float(window[1].attrib['x'])
        self.window['ymax'] = float(window[1].attrib['y'])

        self.selecionar_algoritmo_clipping_retas()
        self.ler_formas(root)

    """
     Função responsável por lêr as formas geometricas e atribuir as suas respectiovas SubClasses
    """
    def ler_formas(self, root):

        for element in root:
            match(element.tag):
                case 'ponto':
                    ponto = Ponto()
                    # A soma feitas as coordenadas é para centralizar o ponto no canvas
                    ponto.x = float(element.attrib['x'])
                    ponto.y = float(element.attrib['y'])
                    ponto.cor = element.attrib['cor']

                    self.formas.append(ponto)

                case 'reta':
                    ponto1 = Ponto()
                    ponto2 = Ponto()
                    # A soma feitas as coordenadas é para centralizar a reta no canvas
                    ponto1.x = float(element[0].attrib['x'])
                    ponto1.y = float(element[0].attrib['y'])

                    ponto2.x = float(element[1].attrib['x'])
                    ponto2.y = float(element[1].attrib['y'])
                    reta = Reta(ponto1,ponto2)
                    reta.cor = element.attrib['cor']

                    self.formas.append(reta)

                case 'poligono':
                    pontos = list()

                    # A soma feitas as coordenadas é para centralizar o poligono no canvas
                    for p in element:
                        ponto = Ponto()
                        ponto.x = float(p.attrib['x'])
                        ponto.y = float(p.attrib['y'])
                        pontos.append(ponto)

                    poligono = Poligono(pontos)
                    poligono.cor = element.attrib['cor']
                    self.formas.append(poligono)

        # Desenhar as formas no canvas
        self.normalizar()
        self.desenhar_formas()
        # self.gerar_arquivo_saida()

    # Função Responsável por desenhar as formas na janela e no monimapa, desenhando de acordo com sua subclasse
    def desenhar_formas(self):
        # Evita que as formas sejam desenhadas uma sobre as outras devido ao Buffer
        self.canvas.delete('all')
        self.minimap.delete('all')

        window = poligonos.Window(self.window_normalizada['xmin'],self.window_normalizada['ymin'],self.window_normalizada['xmax'],self.window_normalizada['ymax'])
        # Aplica o clipping nas formas antes de exibi-las
        for forma in self.formas:

            if isinstance(forma, Ponto):
                forma_clippada_ponto = poligonos.PointClipping(window,forma)
                if forma_clippada_ponto.visible:
                    self.desenhar_ponto(forma_clippada_ponto)
                self.desenhar_ponto_minimapa(forma)

            if isinstance(forma, Reta):
                if self.algClippingReta == 0:
                    forma_clippada_reta = reta_cohen.CohenSutherlandClipping(window,forma)
                elif self.algClippingReta == 1:
                    forma_clippada_reta = reta_liang.LiangBarskyClipping(window,forma)
                if forma_clippada_reta.visible:
                    self.desenhar_reta(forma_clippada_reta)
                self.desenhar_reta_minimapa(forma)

            if isinstance(forma, Poligono):
                forma_clippada_poligono = poligonos.WeilerAthertonPolygonClipping(window,forma)
                if forma_clippada_poligono.visible:
                    self.desenhar_poligono(forma_clippada_poligono)
                self.desenhar_poligono_minimapa(forma)

        self.desenhar_retangulo_minimapa()

    """
    Função destinada a aplicar a transformada de viewport a um ponto e retornar o ponto calculado de acordo com a window e viewport
    """
    def window2viewport(self, ponto, window, viewport):
        ponto_transformado_viewport = Ponto()
        ponto_transformado_viewport.x = ((ponto.x_norm - window['xmin']) / (window['xmax'] - window['xmin']) ) * (viewport['xmax'] - viewport['xmin'])
        ponto_transformado_viewport.y = (1 - (ponto.y_norm - window['ymin']) / (window['ymax'] -window['ymin']) ) * (viewport['ymax'] - viewport['ymin'])
        return ponto_transformado_viewport

    def window2minimap(self, ponto, window, minimap_viewport):
        ponto_transformado_viewport = Ponto()
        ponto_transformado_viewport.x = (((ponto.x)- window['xmin']) / (window['xmax'] - window['xmin']) ) * (minimap_viewport['xmax'] - minimap_viewport['xmin'])
        ponto_transformado_viewport.y = (1 - ((ponto.y) - window['ymin']) / (window['ymax'] -window['ymin']) ) * (minimap_viewport['ymax'] - minimap_viewport['ymin'])

        return ponto_transformado_viewport

    """
     Funcão destinada a desenhar os pontos na janela e no minimapa
    """
    def desenhar_ponto(self, ponto):
        # Desenhar o ponto na janela
        p1 = Ponto()
        p1 = self.window2viewport(ponto, self.window_normalizada, self.viewport)
        raio = 3

        self.canvas.create_oval(p1.x - raio, p1.y - raio, p1.x + raio, p1.y + raio, fill=ponto.cor, outline='')

    def desenhar_ponto_minimapa(self, ponto):
        # Desenhar o ponto no minimapa
        p1 = self.window2minimap(ponto, self.world_size, self.minimap_viewport)
        self.minimap.create_oval(p1.x, p1.y, p1.x + 2, p1.y + 2, fill=ponto.cor, outline='')

    """
    Função destinada a desenhar as retas na janela
    """
    def desenhar_reta(self, reta):
        # Desenhar a reta na janela
        ponto1 = self.window2viewport(reta.ponto1, self.window_normalizada, self.viewport)
        ponto2 = self.window2viewport(reta.ponto2, self.window_normalizada, self.viewport)
        self.canvas.create_line(ponto1.x, ponto1.y, ponto2.x, ponto2.y, fill=reta.cor)

    def desenhar_reta_minimapa(self,reta):
        # Desenhar a reta no minimapa
        ponto1 = self.window2minimap(reta.ponto1, self.world_size, self.minimap_viewport)
        ponto2 = self.window2minimap(reta.ponto2, self.world_size, self.minimap_viewport)
        self.minimap.create_line(ponto1.x, ponto1.y, ponto2.x, ponto2.y, fill=reta.cor)

    """
    Função destinada a desenhar os poligonos na janela
    """
    def desenhar_poligono(self, poligono):
        pontos = poligono.pontos.copy()
        # Desenhar o poligono na janela
        for i in range(len(pontos)):
            pontos[i] = self.window2viewport(poligono.pontos[i],self.window_normalizada,self.viewport)
        self.canvas.create_polygon(tuple((ponto.x, ponto.y) for ponto in pontos), fill='', outline=poligono.cor)

    def desenhar_poligono_minimapa(self, poligono):
        pontos = poligono.pontos.copy()
        for i in range(len(pontos)):
            pontos[i] = self.window2minimap(poligono.pontos[i],self.world_size,self.minimap_viewport)
        self.minimap.create_polygon(tuple((ponto.x, ponto.y) for ponto in pontos), fill='', outline=poligono.cor)

    """
    Desenha o retangulo referente a visão da janela principal no minimapa
    """
    def desenhar_retangulo_minimapa(self):
        pontoA = Ponto(self.window['xmin'],self.window['ymin'])
        pontoB = Ponto(self.window['xmax'],self.window['ymax'])

        pontoA = self.window2viewport(pontoA, self.world_size, self.minimap_viewport)
        pontoB = self.window2viewport(pontoB, self.world_size, self.minimap_viewport)

        self.minimap.create_rectangle(pontoA.x, pontoA.y, pontoB.x, pontoB.y, outline='green', dash=(1,1))

    """
    Função responsável por movimentar a janela principal, permitindo o movimento com as setas do teclado
    """
    def movimentar(self, event):
        if self.arquivo:
            match(event):
                case "Up":
                    self.window['ymin'] = self.window['ymin'] + 0.5
                    self.window['ymax'] = self.window['ymax'] + 0.5
                case "Down":
                    self.window['ymin'] = self.window['ymin'] - 0.5
                    self.window['ymax'] = self.window['ymax'] - 0.5
                case "Left":
                    self.window['xmin'] = self.window['xmin'] - 0.5
                    self.window['xmax'] = self.window['xmax'] - 0.5
                case "Right":
                    self.window['xmin'] = self.window['xmin'] + 0.5
                    self.window['xmax'] = self.window['xmax'] + 0.5
            self.normalizar(self.angulo)
            self.desenhar_formas()
            self.desenhar_retangulo_minimapa()

    """
    Função responsável por rotacionar as formas na janela principal
    Rotaciona de acordo com o angulo passado como parametro
    """
    def rotacionar(self, angulo):
        if self.arquivo:
            self.angulo = self.angulo + angulo
            if self.angulo == 360 or self.angulo == -360:
                self.angulo = 0

            self.normalizar(angulo=self.angulo)
            self.desenhar_formas()
    """
    Função responsável por ajustar o zoom na janela principal
    """
    def aplicar_zoom(self, escala=1.0):
        self.escala = self.escala + escala
        self.normalizar(self.angulo)
        self.desenhar_formas()

    """
    Função responsável por gerar o arquivo de saída com as coordenadas transformadas e centralizadas, adiciona tambem a informaçao da viewport e do World Size
    """
    def gerar_arquivo_saida(self):

        root = ET.Element('dados')
        viewport = ET.SubElement(root,'viewport')

        ET.SubElement(viewport,'vpmin', x= str(self.viewport['xmin']), y= str(self.viewport['ymin']))
        ET.SubElement(viewport,'vpmax', x= str(self.viewport['xmax']), y= str(self.viewport['ymax']))

        window = ET.SubElement(root,'window')

        ET.SubElement(window,'wmin', x= str(self.window['xmin']), y= str(self.window['ymin']))
        ET.SubElement(window,'wmax', x= str(self.window['xmax']), y= str(self.window['ymax']))

        world_size = ET.SubElement(root,'world_size')

        ET.SubElement(world_size,'wsmin', x= str(self.world_size['xmin']), y= str(self.world_size['ymin']))
        ET.SubElement(world_size,'wsmax', x= str(self.world_size['xmax']), y= str(self.world_size['ymax']))

        for forma in self.formas:
            if isinstance(forma, Ponto):
                ponto = ET.SubElement(root,'ponto', x= str(forma.x), y= str(forma.y))

            if isinstance(forma, Reta):
                reta = ET.SubElement(root,'reta')
                ponto1 = ET.SubElement(reta,'ponto', x= str(forma.ponto1.x), y= str(forma.ponto1.y))
                ponto2 = ET.SubElement(reta,'ponto', x= str(forma.ponto2.x), y= str(forma.ponto2.y))

            if isinstance(forma, Poligono):
                poligono = ET.SubElement(root,'poligono')
                for ponto in forma.pontos:
                    ponto = ET.SubElement(poligono,'ponto', x= str(ponto.x), y= str(ponto.y))


        tree = ET.ElementTree(root)
        tree.write('saida.xml')

    """
    Função responsável por chamar as funções de calculo da matriz de normalizacao e as funções de normalizacao das formas geometricas e da window
    """
    def normalizar(self, angulo=0):
        matriz_normalizada = self.calcular_matriz_normalizada(angulo)
        if not self.calculou_window:
            self.normalizar_window(matriz_normalizada)
            self.calculou_window = True
        self.normalizar_formas(matriz_normalizada)

    """
    Função responsável por calcular a matriz de normalização de acordo com o angulo passado como parametro
    """
    def calcular_matriz_normalizada(self, angulo=0):

        matriz_s1 = [
            [self.escala,0,0],
            [0,self.escala,0],
            [0,0,1],
        ]

        matriz_s = [
            [2.0/((self.window['xmax'] - self.window['xmin'])) , 0, 0],
            [0, 2.0/(self.window['ymax'] - self.window['ymin']), 0],
            [0, 0, 1]
        ]
        angulo_radianos = math.radians(-angulo)

        matriz_r = [
            [math.cos(angulo_radianos), -math.sin(angulo_radianos), 0],
            [math.sin(angulo_radianos), math.cos(angulo_radianos), 0],
            [0, 0, 1]
        ]

        window_centro_x = (self.window['xmax'] + self.window['xmin'])/2.0
        window_centro_y = (self.window['ymax'] + self.window['ymin'])/2.0

        matriz_t = [
            [1, 0, -window_centro_x],
            [0, 1, -window_centro_y],
            [0, 0, 1]
        ]

        matriz_normalizada = np.dot(matriz_s1, matriz_s)
        matriz_normalizada = np.dot(matriz_normalizada, matriz_r)
        matriz_normalizada = np.dot(matriz_normalizada, matriz_t)

        return matriz_normalizada

    """
    Função responsável por normalizar a window de acordo com a matriz de normalização
    """
    def normalizar_window(self, matriz_normalizada):
        windowp1 = [
                    [self.window['xmax']],
                    [self.window['ymax']],
                    [1]
                ]

        windowp2 = [[self.window['xmin']],
                    [self.window['ymin']],
                    [1]]

        windowp1 = np.dot(matriz_normalizada, windowp1)
        windowp2 = np.dot(matriz_normalizada, windowp2)

        self.window_normalizada['xmin'] = float(windowp2[0,0])
        self.window_normalizada['ymin'] = float(windowp2[1,0])

        self.window_normalizada['xmax'] = float(windowp1[0,0])
        self.window_normalizada['ymax'] = float(windowp1[1,0])

    """
    Função responsável por normalizar as formas geometricas de acordo com a matriz de normalização
    """
    def normalizar_formas(self, matriz_normalizada):
        for forma in self.formas:
            forma.normalizar(matriz_normalizada)


if __name__ == "__main__":
    root = tk.Tk()
    app = Visualizador(root)
    root.mainloop()
